In [ ]:
# import ee
# import folium
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
# import time

# import subprocess
# try:
#     import geemap
# except ImportError:
#     subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
#     import geemap

!pip install -q geopandas
import geopandas as gpd

!pip install -q altair
import altair as alt

     |████████████████████████████████| 1.0 MB 27.3 MB/s 
     |████████████████████████████████| 15.4 MB 67.5 MB/s 
     |████████████████████████████████| 6.3 MB 48.6 MB/s 


# NEED TO CLEAN UP

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def boundsBuffer(x, buffer=0.075):
    """
    Returns a Polygon geometry that represents a bounding box with a default 7.5% buffer
    """
    minx, miny, maxx, maxy = x
    buffer_x, buffer_y = np.abs(buffer*(maxx-minx)), np.abs(buffer*(maxy-miny))
    minx -= buffer_x
    maxx += buffer_x
    miny -= buffer_y
    maxy += buffer_y
    
    coords = ((minx, miny), (minx, maxy), (maxx, maxy), (maxx, miny))
    return Polygon(coords)

def fireSizeCode(x):
    if (10000 <= x < 50000):
        return "H"
    elif (50000 <= x < 100000):
        return "I"
    elif (100000 <= x < 500000):
        return "J"

In [ ]:
# Using EPSG:4326 for plotting
crs = 'EPSG:4326'

# CA counties
counties = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Oscar/data/CA_Counties/CA_Counties_TIGER2016.shp")
counties.to_crs(crs, inplace=True)

sfLowerBound = counties[counties["NAME"]=="San Francisco"]["geometry"].bounds["maxy"].values[0]

norCal = counties.bounds.apply(lambda x: x[3]>sfLowerBound, axis=1)
norCalCounties = counties[norCal]

# CA wildfires 2013-2020
firesDf = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Anthony/caFirePerimeters_2013_2020.geojson")
firesDf.to_crs(crs, inplace=True)

firesDf = firesDf[(firesDf["GIS_ACRES"] >= 10000) & (~firesDf["CONT_DATE"].isna())]
firesDf["GIS_ACRES"] = firesDf["GIS_ACRES"].round(2)

norCalFires = firesDf.bounds.apply(lambda x: x[1]>sfLowerBound, axis=1)
firesDf = firesDf[norCalFires].reset_index(drop=True)

In [ ]:
# All fires > 10000 acres from 2013-2020 in Northern California
alt.layer(alt.Chart(norCalCounties
            ).mark_geoshape(fill="#E6E6E6", stroke="black"
            ).encode(tooltip=["NAME"]),
          alt.Chart(firesDf[~firesDf["FIRE_NAME"].isin(["AUGUST COMPLEX FIRES", "NORTH COMPLEX"])]
            ).mark_geoshape(stroke="black"
            ).encode(color="GIS_ACRES:Q",
                     tooltip=["FIRE_NAME", "ALARM_DATE", "CONT_DATE", "REPORT_AC", "GIS_ACRES"])
   ).properties(width=700, height=700)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
fireSubset = firesDf["FIRE_NAME"].apply(lambda x: x in ["ATLAS","BALD","BUCK","BUTTE","CALDWELL","CAMP","CARR ",
                                                        "CASCADE", "COVE","FRYING PAN","HAPPY","KINCADE",
                                                        "KING","OAK","REDWOOD VALLEY","ROCKY  ","WALKER"])

subsetDf = firesDf[fireSubset].sort_values(by="FIRE_NAME")
subsetDf["sizeCode"] = subsetDf["GIS_ACRES"].apply(fireSizeCode)

# bounding box as its own df/layer
bboxDf = gpd.GeoDataFrame(subsetDf.loc[:, ~(subsetDf.columns=="geometry")],
                          geometry=subsetDf["geometry"].apply(lambda x: boundsBuffer(x.bounds).difference(x)))

boundsDf = gpd.GeoDataFrame(subsetDf.loc[:, ~(subsetDf.columns=="geometry")],
                            geometry=subsetDf["geometry"].apply(lambda x: boundsBuffer(x.bounds)))

# subsetDf.to_crs("EPSG:3310", inplace=True)
# bboxDf.to_crs("EPSG:3310", inplace=True)
# boundsDf.to_crs("EPSG:3310", inplace=True)

In [ ]:
# subsetDf.to_file("/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/burned/fireBounds.shp")  # fire geometry
# bboxDf.to_file("/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/unburned/bbox.shp")   # differenced bounding box with buffer
# boundsDf.to_file("/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/bounds/bounds.shp")   # bounding box with buffer

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Selected fires with bounding box + 20% buffer
alt.layer(alt.Chart(norCalCounties
            ).mark_geoshape(stroke="black", fill="#E6E6E6"
            ).encode(tooltip=["NAME"]),
          alt.Chart(bboxDf).mark_geoshape(stroke="red", fill="pink"),
          alt.Chart(bounds_df).mark_geoshape(stroke="blue", fill="lightblue"),
          alt.Chart(subsetDf
            ).mark_geoshape(stroke="red"
            ).encode(color="GIS_ACRES:Q",
                     tooltip=["FIRE_NAME", "ALARM_DATE", "CONT_DATE",
                              "REPORT_AC", "GIS_ACRES", "sizeCode"]),
   ).properties(width=700, height=700)

Output hidden; open in https://colab.research.google.com to view.

# Image selection

In [ ]:
import numpy as np
import ee
import folium
from datetime import datetime, timedelta
import time
import subprocess
try:
    import geemap
except ImportError:
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    import geemap

!pip install -q geopandas
import geopandas as gpd

     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 15.4 MB 14.7 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=IVA3o02noE7mylyynFSlfFTsr5deqxneFr9TZfZsUyY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWg6oyC-EFNqmQhaymv91bP4IqYMgA_wvPVNHPbai7v2HO2txOavR-A

Successfully saved authorization token.


In [ ]:
ee.Initialize()

In [ ]:
def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

def convertUnix(x):
    """
    Converts EE.Date to Y-M-D formst
    """
    if isinstance(x, ee.Date):
        x = x.getInfo()["value"]
    
    return datetime.utcfromtimestamp(x/1000).strftime("%Y-%m-%d")# %H:%M:%S')

def mosaicByDate(collection):
    ts = list(map(convertUnix, collection.reduceColumns(reducer=ee.Reducer.toList(),
                                                        selectors=["system:time_start"]
                                        ).get('list').getInfo()))

    # unique imageCollection dates in order
    ts = ee.List(list({i: None for i in ts}.keys()))
    
    return ts.map(lambda x: collection.filterDate(ee.Date(x),
                                                  ee.Date(x).advance(1, "day")
                                     ).mosaic(
                                     ).set("mosaicDate", x))

In [ ]:
bounds_df = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/bounds/bounds.shp")

In [ ]:
# bounds_df.sort_values(by="GIS_ACRES", ascending=False)

,OBJECTID,YEAR_,STATE,AGENCY,UNIT_ID,FIRE_NAME,INC_NUM,ALARM_DATE,CONT_DATE,CAUSE,COMMENTS,REPORT_AC,GIS_ACRES,C_METHOD,OBJECTIVE,FIRE_NUM,SHAPE_Leng,SHAPE_Area,sizeCode,geometry,pre-fire image date,post-fire image date
6,42113,2018,CA,CDF,SHU,CARR,00007808,2018-07-23T00:00:00+00:00,2018-09-01T00:00:00+00:00,10.0,None,229651.0,229651.41,1.0,1.0,None,4.075826,0.099027,J,"POLYGON ((-122.81319 40.45893, -122.81319 40.9...",2018-07-10,2018-10-14
5,42055,2018,CA,CDF,BTU,CAMP,00016737,2018-11-08T00:00:00+00:00,2018-11-26T00:00:00+00:00,11.0,None,153336.0,153335.56,1.0,1.0,None,3.192826,0.065212,J,"POLYGON ((-121.80972 39.57614, -121.80972 39.9...",2018-10-07,2018-12-26
9,39803,2014,CA,USF,KNF,FRYING PAN,006008,2014-08-11T00:00:00+00:00,2014-09-30T00:00:00+00:00,1.0,Happy Camp Cpx #5956; Merged Delta/Faulkstein/Man,133284.2,133177.38,3.0,1.0,None,3.169414,0.058296,J,"POLYGON ((-123.41733 41.47370, -123.41733 41.8...",2014-07-15,2014-10-03
12,39756,2014,CA,USF,ENF,KING,023461,2014-09-13T00:00:00+00:00,2014-10-10T00:00:00+00:00,7.0,<Null>,97717.0,97684.55,3.0,1.0,None,2.951633,0.041050,I,"POLYGON ((-120.72606 38.73190, -120.72606 39.1...",2014-09-03,2014-10-21
13,41885,2017,CA,USF,KNF,OAK,00006016,2017-08-11T00:00:00+00:00,2017-10-10T00:00:00+00:00,1.0,CONT_DATE est frm Inciweb/IRWIN - Eclipse Comp...,45403.0,91125.33,3.0,1.0,None,3.714010,0.039910,I,"POLYGON ((-123.79187 41.61394, -123.79187 41.8...",2017-07-14,2017-10-18
4,21756,2020,CA,NPS,BNP,CALDWELL,00000479,2020-07-22T00:00:00+00:00,2020-09-01T00:00:00+00:00,1.0,None,80859.0,81224.37,7.0,1.0,None,1.832427,0.035573,I,"POLYGON ((-121.62868 41.61286, -121.62868 41.8...",2020-07-15,2020-09-01
11,42478,2019,CA,CDF,LNU,KINCADE,00019376,2019-10-23T00:00:00+00:00,2019-11-10T00:00:00+00:00,11.0,None,NaN,77762.14,1.0,1.0,None,1.924482,0.032577,I,"POLYGON ((-122.91525 38.50629, -122.91525 38.8...",2019-10-01,2019-11-18
3,40705,2015,CA,CDF,AEU,BUTTE,024918,2015-09-10T00:00:00+00:00,2015-09-28T00:00:00+00:00,14.0,None,70868.0,70846.53,1.0,1.0,None,1.587552,0.029513,I,"POLYGON ((-120.73892 38.14103, -120.73892 38.3...",2015-09-06,2015-10-24
15,40738,2015,CA,CDF,LNU,ROCKY,006984,2015-07-29T00:00:00+00:00,2015-08-14T00:00:00+00:00,9.0,"$46,200,000.00 Milion (Cost)",69636.0,69438.16,3.0,1.0,None,1.389513,0.029198,I,"POLYGON ((-122.58873 38.83067, -122.58873 39.0...",2015-07-27,2015-08-19
10,40935,2015,CA,USF,SHF,HAPPY,002066,2015-07-30T00:00:00+00:00,2015-08-08T00:00:00+00:00,1.0,GeoMAC says River Comlpex #2066 Individual fir...,1800.0,68095.24,3.0,1.0,00000149,2.270717,0.029455,I,"POLYGON ((-123.50461 40.78113, -123.50461 41.0...",2015-07-25,2015-09-20


In [ ]:
# # # pre-fire image selection
# for fireName, start, geometry in bounds_df[["FIRE_NAME", "ALARM_DATE", "geometry"]].values[4:6]:
#     # fireOutline = ee.Image().byte().paint(featureCollection=fireBounds.filter(ee.Filter.eq("FIRE_NAME", fireName)),
#     #                                                 color=2,
#     #                                                      width=2
#     #                                              ).visualize(palette="000000")
#     geometry = ee.Geometry.Rectangle(list(geometry.bounds))
#     center = geometry.centroid().getInfo()["coordinates"][::-1]

#     l8_sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"
#              ).filterBounds(geometry
#              ).filterDate(ee.Date(start).advance(-48, "day"), start
#              ).sort("system:time_start")
             
#     l8_sr = mosaicByDate(l8_sr)
    
#     m = folium.Map(location=center, zoom_start=11)

#     for i in range(l8_sr.size().getInfo()):
#         image = ee.Image(l8_sr.get(i)).clip(geometry)#.blend(fireOutline.select("vis-red"))
#         m.add_ee_layer(image,
#                        {"bands": ["SR_B4", "SR_B3", "SR_B2"], 
#                         "gamma": [0.95, 1.1, 1],
#                         "min": 1000, "max": 35000},
#                        image.get("mosaicDate").getInfo())
        
#     m.add_child(folium.LayerControl())
#     print(fireName, start)
#     display(m)
#     print("\n \n")

In [ ]:
# # # post-fire image selection
# for fireName, start, geometry in bounds_df[["FIRE_NAME", "CONT_DATE", "geometry"]].values[4:6]:
    # fireOutline = ee.Image().byte().paint(featureCollection=fireBounds.filter(ee.Filter.eq("FIRE_NAME", fireName)),
    #                                                 color=2,
    #                                                      width=2
    #                                              ).visualize(palette="000000")
    
#     geometry = ee.Geometry.Rectangle(list(geometry.bounds))
#     center = geometry.centroid().getInfo()["coordinates"][::-1]

#     l8_sr = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"
#              ).filterBounds(geometry
#              ).filterDate(ee.Date(start).advance(-16, "day"), ee.Date(start).advance(60, "day")
#              ).sort("system:time_start")
    
#     l8_sr = mosaicByDate(l8_sr)
    
#     m = folium.Map(location=center, zoom_start=11)

#     for i in range(l8_sr.size().getInfo()):
#         image = ee.Image(l8_sr.get(i)).clip(geometry)#.blend(fireOutline.select("vis-red"))
#         m.add_ee_layer(image,
#                        {"bands": ["SR_B4", "SR_B3", "SR_B2"], 
#                         "gamma": [0.95, 1.1, 1],
#                         "min": 1000, "max": 35000},
#                        image.get("mosaicDate").getInfo())
        
#     m.add_child(folium.LayerControl())
#     print(fireName, start)
#     display(m)
#     print("\n \n")


Fire Name | Alarm Date | Contained Date | Pre-Fire Image Date | Post-Fire Image Date
-------------------|------------------|------------------|------------------|------------------
**Atlas** | 10/8/17 | 11/1/17 | 10/4/17 | 11/5/17
**Bald** | 7/30/14 | 8/15/14 | 7/24/14 | 9/10/14
**Buck** | 9/12/17 | 10/30/17 | 8/24/17 | 12/14/17
**Butte** | 9/10/15 | 9/28/15 | 9/6/15 | 10/24/15
**Caldwell** | 7/22/20 | 9/1/20 | 7/15/20 | 2020-09-01
**Camp** | 11/8/18 | 11/26/18 | 10/7/18 | 12/26/18
**Carr** | 7/23/18 | 9/1/18 | 7/10/18 | 10/14/18
**Cascade** | 10/8/17 | 10/26/17 | 10/4/17 | 11/5/17
**Cove** | 7/25/17 | 9/6/17 | 7/16/17 | 10/4/17
**Frying Pan** | 8/11/14 | 9/30/14 | 7/15/14 | 10/3/14
**Happy** | 7/30/15 | 8/8/15 | 7/25/15 | 9/20/15
**Kincade** | 10/23/19 | 11/10/19 | 10/1/19 | 11/18/19
**King** | 9/13/14 | 10/10/14 | 9/3/14 | 10/21/14
**Oak** | 8/11/17 | 10/10/17 | 7/14/17 | 10/18/17
**Redwood Valley** | 10/8/17 | 10/25/17 | 9/25/17 | 10/27/17
**Rocky** | 7/29/15 | 8/14/15 | 7/27/15 | 8/19/15
**Walker** | 9/4/19 | 1/15/20 | 9/1/19 | 2/24/20








In [ ]:
bounds_df["pre-fire image date"] = ["2017-10-04", "2014-07-24", "2017-08-24", "2015-09-06", "2020-07-15", "2018-10-07",
                                    "2018-07-10", "2017-10-04", "2017-07-16", "2014-07-15", "2015-07-25", "2019-10-01",
                                    "2014-09-03", "2017-07-14", "2017-09-25", "2015-07-27", "2019-09-01"]

bounds_df["post-fire image date"] = ["2017-11-05", "2014-09-10", "2017-12-14", "2015-10-24", "2020-09-01", "2018-12-26",
                                     "2018-10-14", "2017-11-05", "2017-10-04", "2014-10-03", "2015-09-20", "2019-11-18",
                                     "2014-10-21", "2017-10-18", "2017-10-27", "2015-08-19", "2020-02-24"]

In [ ]:
fireCoords_df = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/burned/sampleCoordinates.shp")
bboxCoords_df = gpd.read_file("/content/drive/Shareddrives/Capstone 2022/Anthony/fire selection/data/unburned/sampleCoordinates.shp")

In [ ]:
# sld_intervals = ('<RasterSymbolizer>' +
#                 '<ColorMap type="intervals" extended="false" >' +
#                     '<ColorMapEntry color="#ffffff" quantity="-500" label="-500"/>' +   
#                     '<ColorMapEntry color="#706c1e" quantity="-75" label="-50" />' +    # brown: vegetation growth
#                     '<ColorMapEntry color="#0ae042" quantity="100" label="100" />' +      # green: unburned
#                     '<ColorMapEntry color="#fff70b" quantity="225" label="225" />' +    # yellow: low severity
#                     '<ColorMapEntry color="#ff641b" quantity="450" label="450" />' +    # red: moderate severity
#                     '<ColorMapEntry color="#a41fd6" quantity="1200" label="1200" />' +  # purple: high severity
#                 '</ColorMap>' +
#                 '</RasterSymbolizer>')